In [1]:
!pip install ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.6 MB/s eta 0:00:00


# Dataset

In [2]:
!unzip -q /content/data.zip -d /content/custom_data_raw

In [3]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Répertoire d'origine pour les images et annotations
raw_data_path_images = "/content/custom_data_raw/images"  # Images sont dans ce dossier
raw_data_path_labels = "/content/custom_data_raw/obj_train_data"  # Annotations dans ce dossier

# Obtenir toutes les images (jpg/png) présentes dans le dossier "images"
images = [f for f in os.listdir(raw_data_path_images) if f.endswith(('.jpg', '.png'))]

# Split train/val (80% train, 20% val)
train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

# Création des dossiers cible pour images et labels
base_path = "/content/custom_data"


os.makedirs(base_path + "/images/train", exist_ok=True)
os.makedirs(base_path + "/images/val", exist_ok=True)
os.makedirs(base_path + "/labels/train", exist_ok=True)
os.makedirs(base_path + "/labels/val", exist_ok=True)

def move_data(image_list, split):
    for img_name in image_list:
        # Déplacer l'image vers le bon dossier
        src_img = os.path.join(raw_data_path_images, img_name)
        dst_img = os.path.join(base_path, f"images/{split}", img_name)
        shutil.copy(src_img, dst_img)

        # Vérifier et déplacer l'annotation .txt correspondante
        txt_name = img_name.rsplit('.', 1)[0] + '.txt'
        txt_src = os.path.join(raw_data_path_labels, txt_name)
        txt_dst = os.path.join(base_path, f"labels/{split}", txt_name)

        if os.path.exists(txt_src):
            shutil.copy(txt_src, txt_dst)
        else:
            print(f"Pas d'annotation pour {img_name}")

# Appliquer le déplacement aux deux splits : train et val
move_data(train_imgs, "train")
move_data(val_imgs, "val")


In [4]:
import os
import cv2
import shutil

# === PARAMÈTRES ===
input_root = "/content/custom_data"
output_root = "/content/split_custom_data"

base_split_path = "/content/split_custom_data"

tile_size = 320
overlap = 0

splits = ["train", "val"]

def yolo_to_bbox(x_center, y_center, w, h, img_w, img_h):
    x1 = int((x_center - w / 2) * img_w)
    y1 = int((y_center - h / 2) * img_h)
    x2 = int((x_center + w / 2) * img_w)
    y2 = int((y_center + h / 2) * img_h)
    return x1, y1, x2, y2

def bbox_to_yolo(x1, y1, x2, y2, tile_w, tile_h):
    x_center = (x1 + x2) / 2 / tile_w
    y_center = (y1 + y2) / 2 / tile_h
    w = (x2 - x1) / tile_w
    h = (y2 - y1) / tile_h
    return x_center, y_center, w, h

# Boucle sur train et val
for split in splits:
    input_img_dir = os.path.join(input_root, f"images/{split}")
    input_lbl_dir = os.path.join(input_root, f"labels/{split}")
    output_img_dir = os.path.join(output_root, f"images/{split}")
    output_lbl_dir = os.path.join(output_root, f"labels/{split}")

    os.makedirs(output_img_dir, exist_ok=True)
    os.makedirs(output_lbl_dir, exist_ok=True)

    for filename in os.listdir(input_img_dir):
        if not filename.lower().endswith(('.jpg', '.png')):
            continue

        basename = os.path.splitext(filename)[0]
        img_path = os.path.join(input_img_dir, filename)
        label_path = os.path.join(input_lbl_dir, f"{basename}.txt")

        img = cv2.imread(img_path)
        img_h, img_w = img.shape[:2]

        annotations = []
        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                for line in f.readlines():
                    parts = line.strip().split()
                    if len(parts) == 5:
                        cls, xc, yc, w, h = map(float, parts)
                        annotations.append((cls, *yolo_to_bbox(xc, yc, w, h, img_w, img_h)))

        step = tile_size - overlap
        tile_id = 0

        for y in range(0, img_h, step):
            for x in range(0, img_w, step):
                tile = img[y:y + tile_size, x:x + tile_size]
                tile_h, tile_w = tile.shape[:2]

                if tile_h < tile_size or tile_w < tile_size:
                    continue

                tile_filename = f"{basename}_{tile_id}.jpg"
                label_filename = f"{basename}_{tile_id}.txt"
                tile_id += 1

                cv2.imwrite(os.path.join(output_img_dir, tile_filename), tile)

                new_annots = []
                for cls, x1, y1, x2, y2 in annotations:
                    if x1 >= x + tile_size or x2 <= x or y1 >= y + tile_size or y2 <= y:
                        continue

                    bx1 = max(0, x1 - x)
                    by1 = max(0, y1 - y)
                    bx2 = min(tile_size, x2 - x)
                    by2 = min(tile_size, y2 - y)

                    if bx2 - bx1 < 5 or by2 - by1 < 5:
                        continue

                    x_c, y_c, w, h = bbox_to_yolo(bx1, by1, bx2, by2, tile_size, tile_size)
                    new_annots.append(f"{int(cls)} {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}")

                with open(os.path.join(output_lbl_dir, label_filename), "w") as f:
                    f.write("\n".join(new_annots))

    print(f"✅ Split terminé pour '{split}' — {len(os.listdir(output_img_dir))} tuiles générées")


✅ Split terminé pour 'train' — 2180 tuiles générées
✅ Split terminé pour 'val' — 532 tuiles générées


In [5]:
# Chemin vers le fichier obj.names exporté par CVAT (modifie si besoin)
obj_names_path = "/content/custom_data_raw/obj.names"

# Lire les noms de classes depuis obj.names
with open(obj_names_path, 'r') as f:
    class_names = [line.strip() for line in f if line.strip()]

# Générer le contenu du fichier YAML avec les augmentations de couleur
yaml_content = f"""
path: {base_split_path}
train: images/train
val: images/val
nc: {len(class_names)}
names: {class_names}
augment: True  # Activer l'augmentation des données
auto_augment: albumentations



# weights: [0.9, 0.1]  # Exemple de pondération si olive est trop présente
"""


# Écriture du fichier data.yaml
with open(f"{base_path}/data.yaml", "w") as f:
    f.write(yaml_content.strip())

with open(f"{base_split_path}/data.yaml", "w") as f:
    f.write(yaml_content.strip())

print(" Fichier data.yaml généré automatiquement :")
print(yaml_content)


 Fichier data.yaml généré automatiquement :

path: /content/split_custom_data
train: images/train
val: images/val
nc: 2
names: ['olive', 'potential_olive']
augment: True  # Activer l'augmentation des données
auto_augment: albumentations



# weights: [0.9, 0.1]  # Exemple de pondération si olive est trop présente



# Import

In [6]:
from ultralytics import YOLO
import os
import torch
import psutil
import time
import numpy as np

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# Config

In [7]:
!unzip -q /content/train.zip -d /content/train

In [8]:
MODEL_PATH = "/content/train/train/weights/best.pt"
DATA_YAML = "/content/split_custom_data/data.yaml"
CONF_THRESHOLD = 0.05
CLASS_NAME = "olive"

# Loading

In [9]:
model = YOLO(MODEL_PATH)

# Inference

In [10]:
start_time = time.time()
metrics = model.val(data=DATA_YAML, split="val", conf=CONF_THRESHOLD, verbose=False)
end_time = time.time()

Ultralytics 8.3.119 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 92 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 13.8MB/s]

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1437.8±550.1 MB/s, size: 52.0 KB)



val: Scanning /content/split_custom_data/labels/val... 532 images, 259 backgrounds, 0 corrupt: 100%|██████████| 532/532 [00:00<00:00, 2038.44it/s]

val: New cache created: /content/split_custom_data/labels/val.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [04:54<00:00,  8.66s/it]


                   all        532       1221      0.362      0.361      0.374       0.22
Speed: 0.6ms preprocess, 547.8ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/val


# Metrics

In [11]:
from ultralytics import YOLO
from pathlib import Path
import cv2
import os
from ultralytics.utils.metrics import bbox_iou

# 🔧 Paramètres
model_path = "/content/train/train/weights/best.pt"
data_path = "/content/split_custom_data/data.yaml"
val_images_dir = "/content/split_custom_data/images/val"
conf_threshold = 0.05  # 🔁 MODIFIE ici le seuil de confiance
target_class = "olive"
iou_threshold = 0.5  # pour comptage TP/FN

# 📦 Charger modèle + noms des classes
model = YOLO(model_path)
names = model.names
target_id = list(names.values()).index(target_class)

# 📂 Liste des fichiers image de validation
image_paths = list(Path(val_images_dir).rglob("*.jpg")) + list(Path(val_images_dir).rglob("*.png"))

# 🔢 Compteurs
TP, FP, FN = 0, 0, 0

# 🔍 Parcourir les images
import torch

# 🔍 Parcourir les images
for img_path in image_paths:
    results = model.predict(source=str(img_path), conf=conf_threshold, iou=iou_threshold, verbose=False)[0]

    # Prédictions pour la classe cible
    preds = [b for b in results.boxes.data.cpu().numpy() if int(b[5]) == target_id]
    pred_boxes = [b[:4] for b in preds]  # x1, y1, x2, y2

    # Ground truth : via les annotations val
    label_path = str(img_path).replace("/images/", "/labels/").rsplit(".", 1)[0] + ".txt"
    gt_boxes = []
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            for line in f:
                cls, x, y, w, h = map(float, line.strip().split())
                if int(cls) == target_id:
                    # YOLO format -> (x1, y1, x2, y2)
                    img = cv2.imread(str(img_path))
                    H, W = img.shape[:2]
                    cx, cy, bw, bh = x * W, y * H, w * W, h * H
                    x1 = cx - bw / 2
                    y1 = cy - bh / 2
                    x2 = cx + bw / 2
                    y2 = cy + bh / 2
                    gt_boxes.append([x1, y1, x2, y2])

    # 🧠 Matching GT ↔ prédictions
    matched_gt = set()
    for pred_box in pred_boxes:
        matched = False
        for i, gt_box in enumerate(gt_boxes):
            # Convertir en tensor avant de calculer l'IoU
            pred_box_tensor = torch.tensor(pred_box).unsqueeze(0)  # Convertir en tensor
            gt_box_tensor = torch.tensor(gt_box).unsqueeze(0)  # Convertir en tensor
            iou = bbox_iou(pred_box_tensor, gt_box_tensor)[0]  # Calcul de l'IoU
            if iou > iou_threshold and i not in matched_gt:
                TP += 1
                matched_gt.add(i)
                matched = True
                break
        if not matched:
            FP += 1
    FN += len(gt_boxes) - len(matched_gt)

# 📊 Calcul des métriques
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0

print(f"\n🔎 Résultats pour la classe '{target_class}' (seuil conf = {conf_threshold}):")
print(f" - Vrais positifs (TP) : {TP}")
print(f" - Faux positifs (FP)  : {FP}")
print(f" - Faux négatifs (FN)  : {FN}")
print(f" - Précision           : {precision:.3f}")
print(f" - Rappel              : {recall:.3f}")




🔎 Résultats pour la classe 'olive' (seuil conf = 0.05):
 - Vrais positifs (TP) : 1024
 - Faux positifs (FP)  : 612
 - Faux négatifs (FN)  : 170
 - Précision           : 0.626
 - Rappel              : 0.858


In [12]:
names = model.names
class_id = list(names.values()).index(CLASS_NAME)

precision = metrics.box.p[class_id]
recall = metrics.box.r[class_id]
map50 = metrics.box.ap50[class_id]
map5095 = metrics.box.ap[class_id]

f1 = 2 * precision * recall / (precision + recall + 1e-8)

print(f"\n🔎 Métriques classe '{CLASS_NAME}' (id {class_id}):")
print(f" - Précision : {precision:.3f}")
print(f" - Rappel    : {recall:.3f}")
print(f" - F1-score  : {f1:.3f}")
print(f" - mAP@0.5   : {map50:.3f}")
print(f" - mAP@0.5:0.95 : {map5095:.3f}")

# === TEMPS D'INFÉRENCE (moyen estimé) ===
inference_time_total = metrics.speed['inference']  # en ms





# === TAILLE DU MODÈLE ===
model_size = os.path.getsize(MODEL_PATH) / (1024 ** 2)
print(f"💾 Taille du modèle : {model_size:.2f} MB")

# === MÉMOIRE GPU ou RAM ===
if torch.cuda.is_available():
    print(f"📊 Mémoire GPU utilisée : {torch.cuda.max_memory_allocated() / (1024**2):.2f} MB")
else:
    print(f"📊 Mémoire RAM utilisée : {psutil.Process().memory_info().rss / (1024**2):.2f} MB")


🔎 Métriques classe 'olive' (id 0):
 - Précision : 0.723
 - Rappel    : 0.722
 - F1-score  : 0.723
 - mAP@0.5   : 0.749
 - mAP@0.5:0.95 : 0.441
💾 Taille du modèle : 49.58 MB
📊 Mémoire RAM utilisée : 1342.45 MB


In [14]:
!pip install sahi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 9.3 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=9b79db62cd9b495a314ea365f5dc72df4e5019b893eb37fb6c51ab68a961e947
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.11.0.86
    Uninstalling opencv-python-4.11.0.86:
      Successfully uninstalled opencv-python-4.11.0.86


In [15]:
from sahi.predict import get_sliced_prediction
from sahi.models.ultralytics import UltralyticsDetectionModel
import os
from pathlib import Path
import torch
import cv2
from ultralytics.utils.metrics import bbox_iou

# === PARAMÈTRES ===
MODEL_PATH = "/content/train/train/weights/best.pt"
DATA_YAML = "/content/split_custom_data/data.yaml"
VAL_IMAGES_DIR = "/content/split_custom_data/images/val"
CONF_THRESHOLD = 0.05
IOU_THRESHOLD = 0.5
CLASS_NAME = "olive"
SLICED_IMAGE_SIZE = 320  # taille des tuiles SAHI
OVERLAP_RATIO = 0.2
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Charger modèle SAHI
detection_model = UltralyticsDetectionModel(
    model_path=MODEL_PATH,
    confidence_threshold=CONF_THRESHOLD,
    device=DEVICE
)

# Obtenir le class_id de la classe cible
model = YOLO(MODEL_PATH)
names = model.names
class_id = list(names.values()).index(CLASS_NAME)

# 🔢 Compteurs
TP, FP, FN = 0, 0, 0

# 🔍 Parcourir les images de validation
image_paths = list(Path(VAL_IMAGES_DIR).rglob("*.jpg")) + list(Path(VAL_IMAGES_DIR).rglob("*.png"))

for img_path in image_paths:
    # === PRÉDICTION SAHI ===
    result = get_sliced_prediction(
        str(img_path),
        detection_model,
        slice_height=SLICED_IMAGE_SIZE,
        slice_width=SLICED_IMAGE_SIZE,
        overlap_height_ratio=OVERLAP_RATIO,
        overlap_width_ratio=OVERLAP_RATIO,
        verbose=False
    )

    # === Récupérer les boxes SAHI ===
    preds = []
    for pred in result.object_prediction_list:
        if pred.category.id == class_id:
            bbox = pred.bbox.to_xyxy()
            preds.append(bbox)

    # === Ground-truths ===
    label_path = str(img_path).replace("/images/", "/labels/").rsplit(".", 1)[0] + ".txt"
    gt_boxes = []
    if os.path.exists(label_path):
        img = cv2.imread(str(img_path))
        H, W = img.shape[:2]
        with open(label_path, "r") as f:
            for line in f:
                cls, x, y, w, h = map(float, line.strip().split())
                if int(cls) == class_id:
                    # YOLO format → xyxy
                    cx, cy, bw, bh = x * W, y * H, w * W, h * H
                    x1 = cx - bw / 2
                    y1 = cy - bh / 2
                    x2 = cx + bw / 2
                    y2 = cy + bh / 2
                    gt_boxes.append([x1, y1, x2, y2])

    # === MATCHING IoU ===
    matched_gt = set()
    for pred_box in preds:
        matched = False
        for i, gt_box in enumerate(gt_boxes):
            pred_tensor = torch.tensor(pred_box).unsqueeze(0)
            gt_tensor = torch.tensor(gt_box).unsqueeze(0)
            iou = bbox_iou(pred_tensor, gt_tensor)[0]
            if iou > IOU_THRESHOLD and i not in matched_gt:
                TP += 1
                matched_gt.add(i)
                matched = True
                break
        if not matched:
            FP += 1
    FN += len(gt_boxes) - len(matched_gt)

# 📊 Calcul des métriques
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1 = 2 * precision * recall / (precision + recall + 1e-8)

print(f"\n🔎 Résultats pour la classe '{CLASS_NAME}':")
print(f" - Vrais positifs (TP) : {TP}")
print(f" - Faux positifs (FP)  : {FP}")
print(f" - Faux négatifs (FN)  : {FN}")
print(f" - Précision           : {precision:.3f}")
print(f" - Rappel              : {recall:.3f}")
print(f" - F1-score            : {f1:.3f}")



🔎 Résultats pour la classe 'olive':
 - Vrais positifs (TP) : 1016
 - Faux positifs (FP)  : 547
 - Faux négatifs (FN)  : 178
 - Précision           : 0.650
 - Rappel              : 0.851
 - F1-score            : 0.737
